## Bootstrap Aggregation of Small Neural Networks
Before utilizing the Twitter data, we will build small neural networks and combining them to an ensemble by bagging, i.e. bootstrap aggregation. For the exploratory data analysis, see the corresponding Jupyter Notebook.

### Loading and Preparing the Data
We start by loading the data and doing some light feature engineering. 

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import preprocessing.numerical_data as pnd
%matplotlib inline

In [ ]:
df = pd.read_csv(os.path.join("data", "raw", "input.csv"), sep=";", index_col=0, parse_dates=[0], dtype=np.float64)
df.info()

We will first apply the transformations that were development during the above mentioned EDA part. In summary, these comprised removing NA values, preventing data leaks by shifting some column values (e.g. shifting the "Settle" columns a day back). Having seen that the price columns seem to follow a log-normal distribution, we transform these by applying the natural logarithm. Please note that I will omit a formal hypothesis test for log-normality, so consider this transformation a heuristic. To avoid issues with logarithms near or at 0, we add 1 to the columns in question. The function $\left[0, \infty\right) \ni x \mapsto \log(1+x)$ maps to $\left[0, \infty\right)$.

In [ ]:
df = pnd.chain_preparations(df)

In [ ]:
loggable_columns= [col for col in df.columns if any(word in col for word in ("Open", "High", "Low", "Settle"))
                   and not "Interest" in col]
df[loggable_columns] = (df[loggable_columns] + 1).apply(np.log)

In [ ]:
pnd.plot_univariate_distributions(df, loggable_columns, ncols=3)

As seen above, the prior assumption of log-normality was justified at least for Gas and Oil columns. Since no obvious seasonality could be seen during the EDA, we will omit season based feature engineering. 

### Splitting and Scaling the Data
In preparation for the training process, we split the data by a 4:1 ratio. Additionally, we transform the data by using a standard scaler, that is, we center the data around mean 0 and scale to achieve variance 1.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

seed = 123
np.random.seed(seed)

cols = ['Open', 'Prev. Day Open Interest', 'Gas.Open',
       'Coal.Open', 'Oil.Open', 'Prev. Day High',
       'Prev. Day Low', 'Prev. Day Settle', 'Prev. Day Change',
       'Prev. Day Gas.High', 'Prev. Day Gas.Low', 'Prev. Day Gas.Settle',
       'Prev. Day Gas.Change', 'Prev. Day Coal.High', 'Prev. Day Coal.Low',
       'Prev. Day Coal.Settle', 'Prev. Day Coal.Change', 'Prev. Day Oil.High',
       'Prev. Day Oil.Low', 'Prev. Day Oil.Settle', 'Prev. Day Oil.Change']

X = df[cols].values
y = df["Settle"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Defining the Model
For the model, we use a relatively small Neural Network consisting of two hidden layers. The number of hidden units will be 16 and 8, respectively. To speed up learning, we apply batch normalization after the first hidden layer. Both hidden activations will be rectified linear units (ReLUs). We will optimize with the Adam optimizer and minimize the mean squared error.

In [ ]:
from sklearn.ensemble import BaggingRegressor
import models.text_free as mtf
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error

Next, we will fit the model to the training data. We will use a small ensemble with only 15 regressors.

In [ ]:
estimator = BaggingRegressor(KerasRegressor(mtf.simple_nn, input_dim=X.shape[1], epochs=500, batch_size=128, verbose=0),
                             n_estimators=15, n_jobs=-1, iid=False)
estimator.fit(X_train, y_train)

Let us assess the model performance according to the mean squared error:

In [ ]:
print("Training MSE:", mean_squared_error(estimator.predict(X_train), y_train))
print("Test MSE:", mean_squared_error(estimator.predict(X_test), y_test))

We observe that test performance is worse than training performance. This is likely happening due to overfitting. The model above has a total of 529 trainable parameters which is more than half of the training samples. Since it is next to impossible to generate more training data at this point, we have to change the model instead of the data. There are two relatively obvious ways to overcome this issue, one being the reduction of model complexity (e.g. less hidden units) and the other one being more regularization (e.g. $L^2$-regularization or dropout).

In [ ]:
estimator.estimators_[0].model.summary()

Consequently, we should make an attempt to reduce the variance of the model. For now, we will stick with dropout with some probability $p \in [0, 1]$ which randomly blocks units from the forward propagation pass. The decision random variables are iid $\sim{}{\rm Bernoulli}(p)$. We will use a line search to find a suitable value for the dropout probability $p$. The cross validation error will, however, still underestimate the true error. This is because we have a subtle data leak in our fitting process. The standard scaler was trained with the whole training set. Thus, the individual models in the cross-validation steps contain information about their respective validation data. To avoid this, one could include the scaling step into the model fitting process. That way, no data is leaked.

Due to an issue with Keras, we will have to manually write the GridSearch at the moment of writing.

In [ ]:
from sklearn.model_selection import GridSearchCV
import keras.backend as K

K.clear_session()

param_grid = {"n_estimators": [15], "base_estimator__dropout_proba": [0.0, 0.1, 0.2, 0.3, 0.4]}

best_model_cv = GridSearchCV(BaggingRegressor(KerasRegressor(mtf.simple_nn, input_dim=X.shape[1], epochs=500, batch_size=128,
                                                             verbose=0), n_jobs=-1), param_grid, cv=5, n_jobs=-1, iid=False) 
best_model_cv.fit(X_train, y_train)
best_model_cv.best_params_

Let us verify that regularization did indeed increase the performance. For this, we will investigate the parameters chosen by the grid search.